In [1]:
import json
import torch
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import sys
import warnings
warnings.filterwarnings('ignore')

# Path to local library
sys.path.append('./models')

# Local library
from util import load_data, normalise, train_validate, test

# Random seed
params = json.load(open("hyperparams.json", "r"))
NJOBS = params['njobs']
RANDOM_STATE = params['random_state']
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE) 
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cpu device


In [2]:
rssi_train, accl_train, location_train, _ = load_data(params['data_path'], control=True)
rssi_test, accl_test, location_test, _ = load_data(params['data_path'], control=False)

train_idx, validation_idx = np.array([]).astype(np.int64), np.array([]).astype(np.int64)
for room in ['dining_room', 'living_room', 'kitchen', 'hall', 'stairs', 'porch/outside_front_door']:
    temp = np.unique(np.argwhere(location_train == room)[:, 0])
    if len(temp):
        # Limit train data to 300 samples per room (= 1 minute worth of samples per room with 5Hz sampling rate)
        limit = np.min([len(temp), 300])
        if limit < 300:
            limit = int(limit / 2)
        train_idx = np.concatenate([train_idx, temp[:limit]])
        validation_idx = np.concatenate([validation_idx, temp[limit:]])
    else:
        print(f'Data for ({room}) is not available!')

rssi_scaler, rssi_train, rssi_test, rssi_val = normalise(rssi_train[train_idx], rssi_test, rssi_train[validation_idx])
accl_scaler, accl_train, accl_test, accl_val = normalise(accl_train[train_idx], accl_test, accl_train[validation_idx])

encoder = OneHotEncoder(sparse=False)
encoder.fit(np.unique(np.concatenate(
    [location_train[train_idx], location_train[validation_idx], location_test]
)).reshape(-1, 1))
location_val = encoder.transform(location_train[validation_idx].reshape(-1, 1)).reshape(len(validation_idx), location_train.shape[1], -1)
location_train = encoder.transform(location_train[train_idx].reshape(-1, 1)).reshape(len(train_idx), location_train.shape[1], -1)
location_test = encoder.transform(location_test.reshape(-1, 1)).reshape(rssi_test.shape[0], rssi_test.shape[1], -1)

print('Train data: {}, Validation data: {}, Test data: {}'.format(
    rssi_train.shape, rssi_val.shape, rssi_test.shape
))

Data for (stairs) is not available!
Data for (porch/outside_front_door) is not available!
Train data: (952, 25, 20), Validation data: (13627, 25, 20), Test data: (14689, 25, 20)


In [3]:
model = train_validate(
    torch.from_numpy(rssi_train).type(torch.float).to(device), 
    torch.from_numpy(accl_train).type(torch.float).to(device), 
    torch.from_numpy(location_train).type(torch.float).to(device),
    torch.from_numpy(rssi_val).type(torch.float).to(device),
    torch.from_numpy(accl_val).type(torch.float).to(device),
    torch.from_numpy(location_val).type(torch.float).to(device), 
    params
)

Batch: 0, Train Loss: 36.59211, Acc: 42.00000
Batch: 500, Train Loss: 33.80576, Acc: 38.33333
Val Acc: 0.41256
Batch: 0, Train Loss: 23.79462, Acc: 79.00000
Batch: 500, Train Loss: 26.36288, Acc: 60.50000
Val Acc: 0.51611
Batch: 0, Train Loss: 23.41401, Acc: 73.00000
Batch: 500, Train Loss: 24.67243, Acc: 63.66667
Val Acc: 0.61077
Batch: 0, Train Loss: 22.74483, Acc: 80.00000
Batch: 500, Train Loss: 22.55278, Acc: 71.00000
Val Acc: 0.68401
Batch: 0, Train Loss: 22.22615, Acc: 85.00000
Batch: 500, Train Loss: 20.45045, Acc: 79.83333
Val Acc: 0.73685
Batch: 0, Train Loss: 21.40193, Acc: 77.00000
Batch: 500, Train Loss: 18.14055, Acc: 84.16667
Val Acc: 0.80487
Batch: 0, Train Loss: 20.42444, Acc: 81.00000
Batch: 500, Train Loss: 16.12555, Acc: 90.83333
Val Acc: 0.84369
Batch: 0, Train Loss: 19.69536, Acc: 82.00000
Batch: 500, Train Loss: 14.37757, Acc: 91.16667
Val Acc: 0.87936
Batch: 0, Train Loss: 19.19303, Acc: 81.00000
Batch: 500, Train Loss: 12.86360, Acc: 94.33333
Val Acc: 0.90805
B

In [4]:
test(
    model, 
    torch.from_numpy(rssi_test).type(torch.float).to(device), 
    torch.from_numpy(accl_test).type(torch.float).to(device), 
    torch.from_numpy(location_test).type(torch.float).to(device), 
    params
)

Test error: 
 Accuracy: 1.2%


1.1845598747361972